In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.csv as pc
import pyarrow.hdfs as hdfs
import pandas as pd
import pyspark
import pandas as pd
from pyspark.sql.functions import col, split, explode
from pyspark.sql import SparkSession
from pyarrow import fs, hdfs

hdfs.HadoopFileSystem("hdfs://namenode:9000")
conexao = fs.HadoopFileSystem(host="namenode",port=9000)

#conexao.create_dir('/guilherme')
#conexao.delete_file('/somativa/sentiment140.csv')
#conexao.delete_dir('/guilherme')
conexao.get_file_info(fs.FileSelector('/', recursive=True))
origem = '/home/jovyan/work/sentiment140.csv'
destino = '/guilherme/sentiment140.csv'


#conexao.copy_file(origem, destino)

with conexao.open_output_stream(destino) as stream:
    stream.write(open(origem,'rb').read())


spark = SparkSession.builder.getOrCreate()
df = spark.read.csv("hdfs://namenode:9000/guilherme/sentiment140.csv", header=True, inferSchema=True)
df.show()

# Tokenização
df_tokens = df.withColumn('tokens', split(col('text'), '\\W+'))
df_tokens = df_tokens.withColumn('tokens', explode(col('tokens')))

# Contar as palavras mais frequentes
df_most_frequent_tokens = df_tokens.groupBy('tokens').count().orderBy('count', ascending=False)

In [1]:
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.csv as pc
import pyarrow.hdfs as hdfs
import pandas as pd
import pyspark
import pandas as pd
from pyspark.sql.functions import col, split, explode
from pyspark.sql import SparkSession
from pyarrow import fs, hdfs

hdfs.HadoopFileSystem("hdfs://namenode:9000")
conexao = fs.HadoopFileSystem(host="namenode",port=9000)

/tmp/ipykernel_181/2085559598.py:12: FutureWarning: pyarrow.hdfs.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  hdfs.HadoopFileSystem("hdfs://namenode:9000")


In [ ]:
conexao.get_file_info(fs.FileSelector('/', recursive=True))

In [2]:
# Deletar o diretório se ele já existir
try:
    conexao.delete_dir('/guilherme')
    conexao.delete_file('/guilherme/sentiment.csv')
except Exception as e:
    print(f"O diretório foi excluido!: {e}")
    
conexao.create_dir('/guilherme')

O diretório foi excluido!: [Errno 2] Calling GetPathInfo for '/guilherme' failed. Detail: [errno 2] No such file or directory


In [3]:
conexao.get_file_info(fs.FileSelector('/', recursive=True))

[<FileInfo for '/guilherme': type=FileType.Directory>]

In [4]:
origem = '/home/jovyan/work/sentiment140.csv'
destino = '/guilherme/sentiment140.csv'

In [5]:
conexao.get_file_info(fs.FileSelector('/', recursive=True))

[<FileInfo for '/guilherme': type=FileType.Directory>]

In [6]:
with conexao.open_output_stream(destino) as stream:
    stream.write(open(origem,'rb').read())

In [5]:
conexao.get_file_info(fs.FileSelector('/', recursive=True))

[<FileInfo for '/guilherme': type=FileType.Directory>,
 <FileInfo for '/guilherme/sentiment140.csv': type=FileType.File, size=44326223>]

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.read.csv("hdfs://namenode:9000/guilherme/sentiment140.csv", header=True, inferSchema=True, sep=';')
df.show()

+------+----------+--------------------+--------+---------------+--------------------+
|target|       ids|                date|    flag|           user|                text|
+------+----------+--------------------+--------+---------------+--------------------+
|     0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|     0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|     0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|     0|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|     0|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|     0|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|     0|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|     0|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|     0|1467811795|Mon Apr 06 22:20:...|NO_

In [9]:
from pyspark.sql.functions import col, lower, regexp_replace, split, explode

# Seleciona a coluna de texto e guarda elas no dataframe tweets
tweets_df = df.select("text")

# Limpa o texto para facilitar entendimento 
tweets_df_cleaned = tweets_df.withColumn("text", lower(regexp_replace(col("text"), "[^a-zA-Z0-9@#\\s]", "")))
# Mostra os primeiros registros para verificar a limpeza
tweets_df_cleaned.show(5, truncate=False)

# Tokenizar os tweets
words_df = tweets_df_cleaned.withColumn("word", explode(split(col("text"), "\\s+")))
# Conta a frequência de cada palavra
word_freq_df = words_df.groupBy("word").count().orderBy(col("count").desc())
# Mostra as palavras mais frequentes
word_freq_df.show(10)

# Separando usuários mencionados (que começam com @) e os coloca em um dataframe 
mentions_df = words_df.filter(words_df.word.startswith("@"))
# Conta a frequência de usuário mencionado
mentions_freq_df = mentions_df.groupBy("word").count().orderBy(col("count").desc())
# Mostra as menções mais frequentes
mentions_freq_df.show(10)

# Separa as hashtags e colocas elas em um dataframe 
hashtags_df = words_df.filter(words_df.word.startswith("#"))
# Conta as hashtags mais frequentes 
hashtags_freq_df = hashtags_df.groupBy("word").count().orderBy(col("count").desc())
# Mostra as hashtags mais frequentes
hashtags_freq_df.show(10, truncate=False)

+---------------------------------------------------------------------------------------------------------+
|text                                                                                                     |
+---------------------------------------------------------------------------------------------------------+
|@switchfoot httptwitpiccom2y1zl  awww thats a bummer  you shoulda got david carr of third day to do it d |
|is upset that he cant update his facebook by texting it and might cry as a result  school today also blah|
|@kenichan i dived many times for the ball managed to save 50  the rest go out of bounds                  |
|my whole body feels itchy and like its on fire                                                           |
|@nationwideclass no its not behaving at all im mad why am i here because i cant see you all over there   |
+---------------------------------------------------------------------------------------------------------+
only showing top 5 rows

+--